# FRA-X measurements with 3rd party devices

If sources or sinks have analog interfaces for control, it is possible to use them for impedance measurement with Thales and the [FRA probe](https://www.zahner.de/products-details/probes/fra-probe).

The manual for the FRA probe explains the use case in detail: [FRA Manual](https://doc.zahner.de/hardware/fra_probe.pdf)

In [84]:
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode, ThalesRemoteScriptWrapper
import time

# Connect Python to the already launched Thales-Software

In [85]:
zenniumConnection = ThalesRemoteConnection()
zenniumConnection.connectToTerm("192.168.2.47", "ScriptRemote")

zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
zahnerZennium.forceThalesIntoRemoteScript()

devel version


'128,ScriptRemote,5,6,0,0'

# Configuration of the FRA operation

The [FRA Manual](https://doc.zahner.de/hardware/fra_probe.pdf) describes the application in detail, in the following lines only the parameterization relevant for Python is discussed.

The FRA mode must be switched off so that the analog interface to the external device can be initialized.

For the device to work, the gains of voltage and current must be set on the interface. These describe how much signal voltage on the analog interface corresponds to what output current or voltage.
Also minimum and maximum voltage and current of the device must be set.

In the example, the FRA interface is initialized for use with a [DELTA SM3300 SM 18-220](https://www.delta-elektronika.nl/en/products/dc-power-supplies-3300w-sm3300-series.html) power supply/source, which has the analog extension. However, this source is only limited suitable for FRA operation.

This source has **18 V** output voltage and **220 A** maximum current. The analog voltage on the interface is **0 V to 5 V**.

This results in the settings of the gains in the following lines of code. It may be, as in this example, that some gains must have a negative sign. This must be tested with the respective device.

The source must be controlled separately so that the analog interface is used, the source is switched on and off or for potentiostatic and galvanostatic. The Thales software only provides the analog signals, everything else must be done via the device interface.



In [86]:
zahnerZennium.enableFraMode()

'OK\r'

In [87]:
zahnerZennium.setFraVoltageMinimum(0)
zahnerZennium.setFraVoltageMaximum(18)
zahnerZennium.setFraCurrentMinimum(0)
zahnerZennium.setFraCurrentMaximum(220)

zahnerZennium.setFraVoltageInputGain(5 * (4 / 5)) # gain * (4 / 5)
zahnerZennium.setFraVoltageInputOffset(-25e-3)

zahnerZennium.setFraVoltageOutputGain(18.0/5.0)
zahnerZennium.setFraVoltageOutputOffset(-0.3)

zahnerZennium.setFraCurrentInputGain(-242)
zahnerZennium.setFraCurrentInputOffset(-15.03)

zahnerZennium.setFraCurrentOutputGain(-220.0/5.0)
zahnerZennium.setFraCurrentOutputOffset(-3)

zahnerZennium.setFraPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
zahnerZennium.setCurrent(0)

'OK\r'

After parameterization the FRA mode can be switched on.

At this point, the external device which receives the target signals via FRA must be switched on and configured accordingly.

The output of the device must **not be switched on** beforehand, as the analog interface has not yet been initialized.

Simple DC measurements are also possible.

In [88]:
for i in range(10):
    zahnerZennium.setCurrent(i)
    time.sleep(0.5)
    print(f"Potential:\t{zahnerZennium.getPotential()}\tV")
    print(f"Current:\t{zahnerZennium.getCurrent()}\tA")

Potential:	0.0005221556	V
Current:	0.04569409	A
Potential:	0.01014815	V
Current:	1.006116	A
Potential:	0.01979393	V
Current:	2.000953	A
Potential:	0.02997681	V
Current:	3.007906	A
Potential:	0.04010656	V
Current:	3.994758	A
Potential:	0.05025365	V
Current:	5.002351	A
Potential:	0.05973741	V
Current:	6.009117	A
Potential:	0.06979417	V
Current:	7.020814	A
Potential:	0.07994695	V
Current:	8.008658	A
Potential:	0.09013592	V
Current:	9.006383	A


Then impedance can be measured like with the internal potentiostat.
The commands for switching the potentiostat on and off are not necessary, since the potentiostat is not used.

In [89]:
zahnerZennium.setEISOutputPath(r"C:\THALES\temp\test1")
zahnerZennium.setEISNaming("counter")
zahnerZennium.setEISCounter(13)
zahnerZennium.setEISOutputFileName("spectra")

zahnerZennium.setCurrent(10)
zahnerZennium.setAmplitude(2)
zahnerZennium.setLowerFrequencyLimit(1)
zahnerZennium.setStartFrequency(1000)
zahnerZennium.setUpperFrequencyLimit(10e3)
zahnerZennium.setLowerNumberOfPeriods(3)
zahnerZennium.setLowerStepsPerDecade(2)
zahnerZennium.setUpperNumberOfPeriods(20)
zahnerZennium.setUpperStepsPerDecade(3)
zahnerZennium.setScanDirection("startToMax")
zahnerZennium.setScanStrategy("single")

zahnerZennium.measureEIS()

zenniumConnection.disconnectFromTerm()
print("finish")

finish
